## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀세니움 사용
- 무선청소기 검색에 사용할 URL
    https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&previousKeyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&checkedInfo=N&volumeType=allvs&page=2&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A
- URL 최소화하면
    https://search.danawa.com/dsearch.php?query=무선청소기&page=2&limit=120&sort=saveDESC&list=list

한페이지가 로드될때 5 초 정도 소요

In [84]:
## 필수 라이브러리 사용등록
from selenium import  webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse # url encode
from tqdm import tqdm, tqdm_notebook
import time

In [75]:
## 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url =r'https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&previousKeyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&checkedInfo=N&volumeType=allvs&page=2&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
driver.get(url)
time.sleep(5.0)

In [76]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [77]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [78]:
# id="productItem19965230" 식으로 전개되므로 productItem 이 아니면 패스
# 제품명 가져오기, 만약 애드스마트 영역이면 예외발생
prodItems[0].select('p.prod_name > a')[0].text.strip()

'에어메이드 스마트포스 AMC-3500'

In [79]:
# 제품스펙 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t','')

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm'

In [80]:
# 제품가격 가져오기
# 책에서는 1위 데이터를 가져오도록 했음.
# 우린 input중 두번째 값이 최소값을 담고 있는 것을 확인
prodItems[0].select('input')[1].get('value')

'197000'

In [81]:
# 제품인트로
prodItems[0].select('p.intro_text')[0].text

IndexError: list index out of range

In [96]:
prodData = []

for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip() # 상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t','') # 상품 스펙목록 가져오기
        price = prodItem.select('input')[1].get('value') # 최저가 가져오기
        prodData.append([prodName, specList, price])
    

    except:
        pass

In [ ]:
# 결과 확인
prodData 

[['에어메이드 스마트포스 AMC-3500',
  '핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm',
  '197000'],
 ['일렉트로룩스 Well Q8 WQ81-3OMBF',
  '핸디스틱청소기 / 무선 / 흡입형 / 2021년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 솔형 / 틈새 / [배터리] 사용시간: 53분(최대) / 충전시간: 4시간30분 / 내장형 / [청소] 싸이클론흡입 / LED라이트 / 먼지통: 0.3L / [부가] 셀프스탠딩 / 헤파필터 / 5단계여과 / 색상: 마호가니브론즈 / 무게: 2.8kg / 크기(가로x세로x깊이): 258ㅌ1105ㅌ140mm',
  '329000'],
 ['삼성전자 비스포크 제트 VS20A957E3',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 210W / 2021년형 / [구성] 거치대: 청정스테이션 / 먼지비움 / 충전 / 스탠드거치 / 먼지비움시간: 20초 / 브러쉬: 바닥 / 물걸레: 일반 / 침구 / 솔형 / 틈새 / 연장관 / 먼지봉투: 1.2L / [배터리] 사용시간: 1시간(최대) / 충전시간: 3시간30분 / 분리형(2개) / 리튬이온 / 2200+2200mAh / 25.2V / [청소] 싸이클론흡입 / 자동물공급 / 물분사 / 디지털인버터모터 / 먼지통: 0.5L / [부가] 대화형알림창 / 디스플레이표시 / 배터리잔량표시 / 워셔블헤파필터 / 5단계여과 / 색상: 미스티화이트 / 무게: 2.5kg / 크기(가로x세로x깊이): 250x930x202mm',
  '592730'],
 ['쿠

##### 1번 페이지에서 상품 정보 추출

##### 여러페이지 크롤링 하는 방법
-URL 
    -https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A


In [89]:
## 검색어,페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [103]:
## 상품정보 추출하는 함수
def getProdItems(prodItems):
    prodData = []

    for prodItem in prodItems:
      try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip() # 상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t','') # 상품 스펙목록 가져오기
        price = prodItem.select('input')[1].get('value') # 최저가 가져오기
        prodData.append([prodName, specList, price])
      except:
          pass

      return prodData


In [90]:
# 검색어에 ' '가 있으면 URL에서 링크 클릭이 안되는 문제 발생 --> urlEncode:parse.quote()
getSearchPageUrl('마이크로 sd카드',5)

'https://search.danawa.com/dsearch.php?query=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&originalQuery=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&previousKeyword=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&checkedInfo=N&volumeType=allvs&page=5&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [91]:
total_page = 10

for page in tqdm(range(1, total_page+1)):
    print('페이지 처리')
    time.sleep(1)


  0%|          | 0/10 [00:00<?, ?it/s]

페이지 처리


 10%|█         | 1/10 [00:01<00:09,  1.00s/it]

페이지 처리


 20%|██        | 2/10 [00:02<00:08,  1.00s/it]

...

 80%|████████  | 8/10 [00:08<00:02,  1.00s/it]

페이지 처리


 90%|█████████ | 9/10 [00:09<00:01,  1.00s/it]

페이지 처리


100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [111]:
## 여러페이지 검색후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
totalPage = 10
prodDataTotal = [] # 최종적으로 저장할 리스트

for page in tqdm(range(1,totalPage +1)):
    # 검색 페이지 이동
    url = getSearchPageUrl(keyword,page)
    driver.get(url)
    # 페이지 로딩이 완료될때까지 5초간 대기
    time.sleep(5)

    # 현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems) # 리스트로 추출하는 함수
    
    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList

0it [00:00, ?it/s]


In [107]:
dfProdData = pd.DataFrame(prodDataTotal)

In [108]:
dfProdData.columns = ['제품명', '제품스펙','최저가']

In [112]:
dfProdData

,제품명,제품스펙,최저가
0,LG전자 오브제컬렉션 코드제로 A9S AX9884,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: ...,873728
1,에어메이드 스마트포스 AMC-3500,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성]...,197000
2,샤오미 미홀 K11,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 스탠드거치 / 2500mAh / ...,117120
3,쿠쿠전자 파워클론 CVC-AE1410,핸디스틱청소기 / 무선 / 흡입형 / 소비전력: 420W / 2022년형 / [구성...,341760
4,비쎌(BISSELL) 크로스웨이브 3.0 맥스 2765S,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 소비전력: 150W / 2021년...,752210
5,다이슨 V11 컴플리트 플러스,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 185AW / 소비전력: 545W /...,1394070
6,블랙파인 YTE-S7,핸디스틱청소기 / 무선 / 흡입형 / [구성] 브러쉬: 바닥 / 솔형 / 틈새 / ...,530300
7,벤하임코리아 퓨리타 심플 플러스 BVC-160RS,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 12000Pa / 소비전력: 160W...,373030
8,파나소닉 MC-SD787,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 22350Pa / 소비전...,527000
9,샤오미 SHUNZAO Z11 맥스,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 150AW / 흡입력: 26000Pa...,286610


In [109]:
dfProdData.to_excel('./data/다나와_무선청소기_결과.xlsx', index=False)